In [14]:
import pandas as pd
new_building_file=pd.read_excel('new_buildings.xlsx', index_col=0, header=0, keep_default_na=False)

In [15]:
#------------------------data structure for MBR-----------------------------
class MBR(object):
    def __init__(self, maxlon, maxlat, minlon, minlat, index):
        self.maxlon=maxlon
        self.minlon=minlon
        self.maxlat=maxlat
        self.minlat=minlat
        self.index=index

In [16]:
#------------------------data structure for Leaf Node-----------------------------
class LeafNode(object):
    def __init__(self, n):
        self.n=n
        self.parent=None
        self.isLeaf=True
        self.LeafCount=n
        self.RECchild=[None for i in range(n)]
        self.maxlon=None
        self.minlon=None
        self.maxlat=None
        self.minlat=None
        
    def initNode(self, MBR):
        self.LeafCount=self.n - 1
        self.RECchild=[None for i in range(self.n)]
        self.RECchild[0]=MBR
        self.maxlon=MBR.maxlon
        self.minlon=MBR.minlon
        self.maxlat=MBR.maxlat
        self.minlat=MBR.minlat
    
    def addMBR_updateNode(self, MBR):
        self.RECchild[self.n - self.LeafCount]=MBR
        self.LeafCount-=1
        if(self.maxlon!=None):
            self.maxlon=max(self.maxlon, MBR.maxlon)
        else:
            self.maxlon=MBR.maxlon
            
        if(self.maxlat!=None):
            self.maxlat=max(self.maxlat, MBR.maxlat)
        else:
            self.maxlat=MBR.maxlat
        
        if(self.minlon!=None):
            self.minlon=min(self.minlon, MBR.minlon)
        else:
            self.minlon=MBR.minlon
            
        if(self.minlat!=None):
            self.minlat=min(self.minlat, MBR.minlat)
        else:
            self.minlat=MBR.minlat

In [17]:
#------------------------data structure for Non Leaf Node-----------------------------
class NonLeafNode(object):
    def __init__(self, d):
        self.d=d
        self.parent=None
        self.isLeaf=False
        self.NonLeafCount=d
        self.NODEchild=[None for i in range(d)]
        self.maxlon=None
        self.minlon=None
        self.maxlat=None
        self.minlat=None
    
    def initNode(self, firstNodeChild):
        self.NonLeafCount=self.d - 1
        self.NODEchild=[None for i in range(self.d)]
        self.NODEchild[0]=firstNodeChild
        self.maxlon=firstNodeChild.maxlon
        self.minlon=firstNodeChild.minlon
        self.maxlat=firstNodeChild.maxlat
        self.minlat=firstNodeChild.minlat
        
    def addNODE_update(self, newNode):
        self.NODEchild[self.d - self.NonLeafCount]=newNode
        self.NonLeafCount-=1

        if(self.maxlon!=None):
            self.maxlon=max(self.maxlon, newNode.maxlon)
        else:
            self.maxlon=newNode.maxlon
            
        if(self.maxlat!=None):
            self.maxlat=max(self.maxlat, newNode.maxlat)
        else:
            self.maxlat=newNode.maxlat
        
        if(self.minlon!=None):
            self.minlon=min(self.minlon, newNode.minlon)
        else:
            self.minlon=newNode.minlon
            
        if(self.minlat!=None):
            self.minlat=min(self.minlat, newNode.minlat)
        else:
            self.minlat=newNode.minlat
        
    def updateNODE(self, newNode):
        if(self.maxlon!=None):
            self.maxlon=max(self.maxlon, newNode.maxlon)
        else:
            self.maxlon=newNode.maxlon
            
        if(self.maxlat!=None):
            self.maxlat=max(self.maxlat, newNode.maxlat)
        else:
            self.maxlat=newNode.maxlat
        
        if(self.minlon!=None):
            self.minlon=min(self.minlon, newNode.minlon)
        else:
            self.minlon=newNode.minlon
            
        if(self.minlat!=None):
            self.minlat=min(self.minlat, newNode.minlat)
        else:
            self.minlat=newNode.minlat

In [18]:
#----------------------general function for constructing R-Tree-------------------------
def insert(root, d, n, newMBR):
    CLeafNode=ChooseLeaf(root, n, newMBR)
    if(CLeafNode.LeafCount>0):
        CLeafNode.addMBR_updateNode(newMBR)
        L=CLeafNode
        LL=None
    else:
        L, LL=SplitNode(CLeafNode, d, n, newMBR)
    
    P, PP=AdjustTree(root, L, LL, d, n)
    
    return P, PP#if P is root and PP is not None, we will detect it and create a new root outside

def SplitNode(Node, d, n, newMBR):
    #keep Node and just update the NODEchild and NonLeafCount attribute
    #Create only one new Node
    if(Node.isLeaf==True):
        #case of spliting Leaf Node
        #here newMBR is a MBR object
        allchild=Node.RECchild+[newMBR]
        
        #choose two entries to be in two splitten nodes
        head_node1, head_node2, remaining=PickSeed(allchild)
        
        #initiate two splitten nodes for LeafCount, RECchild, lon&lat
        Node.initNode(head_node1)
        newNode=LeafNode(n)
        newNode.initNode(head_node2)
        
        #add remaining MBR to these two splitten node
        for i in range(n-1):
            tmpEnlarge1=CalEnlarge(Node, remaining[i])
            tmpEnlarge2=CalEnlarge(newNode, remaining[i])
            if(tmpEnlarge1<tmpEnlarge2):
                Node.addMBR_updateNode(remaining[i])
            else:
                newNode.addMBR_updateNode(remaining[i])
    else:
        #case of spliting Non Leaf Node
        #here newMBR is a LeafNode or NonLeafNode object
        allchild=Node.NODEchild+[newMBR]
        
        #choose two entries to be in two splitten nodes
        head_node1, head_node2, remaining=PickSeed(allchild)
        
        #initiate two splitten nodes for NonLeafCount, NODEchild, lon&lat
        Node.initNode(head_node1)
        head_node1.parent=Node
        
        newNode=NonLeafNode(d)
        newNode.initNode(head_node2)
        head_node2.parent=newNode
        
        #add remaining MBR to these two splitten node
        for i in range(d-1):
            tmpEnlarge1=CalEnlarge(Node, remaining[i])
            tmpEnlarge2=CalEnlarge(newNode, remaining[i])
            if(tmpEnlarge1<tmpEnlarge2):
                Node.addNODE_update(remaining[i])
                remaining[i].parent=Node
            else:
                newNode.addNODE_update(remaining[i])
                remaining[i].parent=newNode
        
    return Node, newNode
    
    
def ChooseLeaf(root, n, newMBR):
    N=root
    while(N.isLeaf==False):
        descN=None
        leastEnlarge=float('inf')
        leng=len(N.NODEchild)
        for i in range(leng):
            if(N.NODEchild[i]!=None):
                tmpEnlarge=CalEnlarge(N.NODEchild[i], newMBR)
                if(tmpEnlarge<leastEnlarge):
                    leastEnlarge=tmpEnlarge
                    descN=N.NODEchild[i]
        if(leastEnlarge==float('inf')):
            descN=LeafNode(n)
            descN.parent=N
            N.NODEchild[0]=descN
            N.NonLeafCount-=1
        N=descN
    return N

def CalEnlarge(Node, newMBR):
    maxlon=max(Node.maxlon, newMBR.maxlon)
    maxlat=max(Node.maxlat, newMBR.maxlat)
    minlon=min(Node.minlon, newMBR.minlon)
    minlat=min(Node.minlat, newMBR.minlat)
    
    enlarge=(maxlon-minlon)*(maxlat-minlat)-(Node.maxlon-Node.minlon)*(Node.maxlat-Node.minlat)
    
    return enlarge

def AdjustTree(root, L, LL, d, n):
    N=L
    NN=LL
    #P=N.parent
    #PP=None
    while(N!=root):
        P=N.parent
        PP=None
        P.updateNODE(N)
        if(NN!=None):
            if(P.NonLeafCount>0):
                #P has room for NN
                for i in range(d):
                    if(P.NODEchild[i]==None):
                        P.NODEchild[i]=NN
                        NN.parent=P
                        P.updateNODE(NN)
                        P.NonLeafCount-=1
                        break
            else:
                #P has no room for NN, so P need to be split
                P, PP=SplitNode(P, d, n, NN)
        N, NN = P, PP
    return N, NN

def PickSeed(allchild):
    leng=len(allchild)
    node1=None
    node2=None
    waste=float('-inf')
    
    for i in range(leng):
        for j in range(i+1,leng):
            op1=allchild[i]
            op2=allchild[j]
            tmp=CalWaste(op1, op2)
            if(tmp>waste):
                waste=tmp
                node1=op1
                node2=op2
                remaining=allchild[:i]+allchild[i+1:j]+allchild[j+1:]
    return node1, node2, remaining

def CalWaste(node1, node2):
    area1=CalArea(node1)
    area2=CalArea(node2)
    totalArea=CalTotalArea(node1, node2)
    
    return totalArea-area1-area2

def CalArea(Node):
    maxlon=Node.maxlon
    maxlat=Node.maxlat
    minlon=Node.minlon
    minlat=Node.minlat
    
    area=(maxlon-minlon)*(maxlat-minlat)
    
    return area

def CalTotalArea(Node, newMBR):
    maxlon=max(Node.maxlon, newMBR.maxlon)
    maxlat=max(Node.maxlat, newMBR.maxlat)
    minlon=min(Node.minlon, newMBR.minlon)
    minlat=min(Node.minlat, newMBR.minlat)
    
    TotalArea=(maxlon-minlon)*(maxlat-minlat)
    
    return TotalArea

In [19]:
#-----------------------------main program-------------------------------
#traverse all object in the dataset, insert each object into the R-Tree and adjust the R-Tree
d=8
n=256

root=NonLeafNode(d)
all_rec_num=len(new_building_file)
for ind in range(all_rec_num):
        curMBR=new_building_file.iat[ind,4]
        curMBR=curMBR[5:-2]
        point_list=curMBR.split(', ')
        for p in range(2):
            point_list[p]=point_list[p].split(' ')
            for pp in range(2):
                point_list[p][pp]=float(point_list[p][pp])
        #print(point_list)
        MBRi=MBR(point_list[0][0],point_list[0][1],point_list[1][0],point_list[1][1], new_building_file.iat[ind,0])
        
        P, PP=insert(root, d, n, MBRi)
        if(P==root and PP!=None):
            #create a new root here and update the root variable
            new_root=NonLeafNode(d)
            new_root.NODEchild[0]=P
            new_root.NODEchild[1]=PP
            new_root.NonLeafCount-=2
            P.parent=new_root
            PP.parent=new_root
            root=new_root

In [20]:
#---------------------------statistic program-----------------------------
class statistic(object):
    def __init__(self):
        self.NonLeaf=0
        self.Leaf=0
        pass

    def CalHeight(self, curNode, layer):
        tmp=0
        if(curNode.isLeaf):
            return layer
        for item in curNode.NODEchild:
            if(item!=None):
                tmp=max(tmp,self.CalHeight(item, layer+1))
        return tmp

    def CalNodes(self, curNode):
        #print('hjh')
        if(curNode==None):
            return
        if(curNode.isLeaf==False):
            self.NonLeaf+=1
            for item in curNode.NODEchild:
                #print('hjh')
                self.CalNodes(item)
        else:
            self.Leaf+=1


myStat=statistic()
print(root.NODEchild)
print('Height : ',myStat.CalHeight(root, 1))
myStat.CalNodes(root)
print('Non Leaf : ', myStat.NonLeaf, ', Leaf : ', myStat.Leaf)

[<__main__.NonLeafNode object at 0x11f901910>, <__main__.NonLeafNode object at 0x10e03c160>, <__main__.NonLeafNode object at 0x10defd130>, <__main__.NonLeafNode object at 0x1227d9640>, <__main__.NonLeafNode object at 0x122c30dc0>, <__main__.NonLeafNode object at 0x1222ec4f0>, None, None]
Height :  10
Non Leaf :  1187 , Leaf :  996


In [25]:
#---------------------------window query program-----------------------------
import pandas as pd

class windowQuery():
    def __init__(self):
        self.new_building_file=pd.read_excel('new_buildings.xlsx', index_col=0, header=0, keep_default_na=False)
        self.n=len(self.new_building_file)
        self.RTree_query_res=0
        self.RTree_query_check_sum=0
    
    def approach1(self, query_rectangle):
        res=0
        check_sum=0
        for ind in range(self.n):
            curMBR=self.new_building_file.iat[ind,4]
            curMBR=curMBR[5:-2]
            point_list=curMBR.split(', ')
            #print(point_list)
            for i in range(2):
                point_list[i]=point_list[i].split(' ')
            curMBRrtp=point_list[0]
            curMBRlbp=point_list[1]
            if(float(curMBRrtp[0])<=query_rectangle[1][0] and float(curMBRrtp[1])<=query_rectangle[1][1] and float(curMBRlbp[0])>=query_rectangle[0][0] and float(curMBRlbp[1])>=query_rectangle[0][1]):
                res+=1
            check_sum+=1
        return res, check_sum
    
    def approach2(self, query_rectangle, root):
        if(root==None):
            return
        if(root.isLeaf==False):
            #check each child node to see whether overlap with the query window
            for item in root.NODEchild:
                if(item!=None):
                    if(self.overlap(item, query_rectangle)):
                        self.approach2(query_rectangle, item)
        else:
            #check each child MBR
            for MBR in root.RECchild:
                if(MBR==None):
                    continue
                self.RTree_query_check_sum+=1
                if(MBR.maxlon<=query_rectangle[1][0] and MBR.maxlat<=query_rectangle[1][1] and MBR.minlon>=query_rectangle[0][0] and MBR.minlat>=query_rectangle[0][1]):
                    self.RTree_query_res+=1
                    
    def overlap(self, node, query_rectangle):
        query_maxlon=query_rectangle[1][0]
        query_maxlat=query_rectangle[1][1]
        query_minlon=query_rectangle[0][0]
        query_minlat=query_rectangle[0][1]
        if(node.minlon<query_maxlon and node.maxlon>query_minlon and node.maxlat>query_minlat and node.minlat<query_maxlat):
            return True
        return False

In [13]:
import time

random_points=[[[114.16226489078436, 22.370471903885278], [114.33590002703735, 22.375186879371633]], [[113.93791546343081, 22.35787679672303], [114.01782410871618, 22.405519818125416]], [[113.99372501995832, 22.24961765932702], [114.02722641095893, 22.30719459127742]], [[113.98994934959073, 22.22015117902022], [113.99074033672426, 22.245450793734264]], [[114.12609255193942, 22.22779481895737], [114.16857788794387, 22.276267692594885]], [[114.01894097796169, 22.363194033357406], [114.08662893696763, 22.472785292961706]], [[114.10665147693905, 22.370883581921905], [114.23397426492734, 22.444538035154036]], [[114.14917290838618, 22.398823582262477], [114.22004450686822, 22.428574974772967]], [[114.18644026427476, 22.336579337296747], [114.30792942453161, 22.478985009295975]], [[113.95160102208226, 22.402849288997682], [114.2354947378698, 22.486510184588507]], [[113.97537760424841, 22.294854503751587], [114.08436646416386, 22.351235984632414]], [[113.98723988133435, 22.21911927811007], [114.18507300418617, 22.25667720914321]], [[113.98188329573911, 22.395706562963404], [114.10971940201871, 22.487066198960623]], [[113.95568041248178, 22.211418232611848], [114.089857876143, 22.298284390470503]], [[113.93626828030834, 22.20053767447553], [114.0011445510055, 22.215593239164907]], [[113.9428917626879, 22.222393874435785], [113.96657094151807, 22.26313706183892]], [[114.09647334292447, 22.1978520529682], [114.15567362854283, 22.279581319840627]], [[113.93127016342511, 22.24029238772986], [113.93133415616886, 22.260583593926818]], [[114.02321648511226, 22.250018214421843], [114.0959096264096, 22.474371132434126]], [[114.03771851971304, 22.297291378040043], [114.07242183379745, 22.39045230103068]]]
#print(len(random_points))
random_points+=[[[114.06068961768129, 22.20311825374936], [114.0685401057541, 22.243116292614456]], [[114.2263067677991, 22.50194523157252], [114.27256422844611, 22.504257943907444]], [[114.09662011266849, 22.201712551554362], [114.15295582081279, 22.21505791884449]], [[113.97728628907608, 22.290878881549364], [114.02748017711606, 22.36942144860932]], [[113.94149059284678, 22.36060959670548], [114.05131237089478, 22.481301553695854]], [[113.9399211336652, 22.372301642577664], [113.97755773588669, 22.491107363091345]], [[114.01419078106989, 22.208832878541543], [114.18822355061339, 22.20941607084293]], [[113.96246748521664, 22.19924415239848], [113.97407558917821, 22.327514291431314]], [[114.28178145342886, 22.28082796669083], [114.29888807398275, 22.299415790585716]], [[113.99004156037108, 22.267773024933838], [114.00486783933198, 22.320389142059653]]]
#print(len(random_points))
wQ=windowQuery()
#----------------------------query of approach 1 ------------------------------
record=[]
time_rec=[]
count=0
for item in random_points:
    #print("The ",count+1," query window is ",item)
    count+=1
    time_single_query=[]
    for i in range(5):
        starttime=time.time()
        res, check_sum=wQ.approach1(item)
        #record.append(res)
        endtime=time.time()
        timespan=round(endtime - starttime, 2)
        time_single_query.append(timespan)
    record.append([res, check_sum])
    print("The number of objects in NO.", count," query window : ",res,', checked polygon number: ',check_sum)
    print("min runtime : ", min(time_single_query), ". max runtime : ", max(time_single_query), ". avg runtime : ", sum(time_single_query)/len(time_single_query), ".")
    #print("The number of polygons checked is ", check_sum)
    #break

The number of objects in NO. 1  query window :  494 , checked polygon number:  76718
min runtime :  1.49 . max runtime :  1.54 . avg runtime :  1.516 .
The number of objects in NO. 2  query window :  2266 , checked polygon number:  76718
min runtime :  1.46 . max runtime :  1.5 . avg runtime :  1.48 .
The number of objects in NO. 3  query window :  761 , checked polygon number:  76718
min runtime :  1.48 . max runtime :  1.53 . avg runtime :  1.514 .
The number of objects in NO. 4  query window :  2 , checked polygon number:  76718
min runtime :  1.43 . max runtime :  1.45 . avg runtime :  1.438 .
The number of objects in NO. 5  query window :  1793 , checked polygon number:  76718
min runtime :  1.44 . max runtime :  1.51 . avg runtime :  1.466 .
The number of objects in NO. 6  query window :  3916 , checked polygon number:  76718
min runtime :  1.43 . max runtime :  1.49 . avg runtime :  1.456 .
The number of objects in NO. 7  query window :  9742 , checked polygon number:  76718
min

In [28]:
import time

random_points=[[[114.16226489078436, 22.370471903885278], [114.33590002703735, 22.375186879371633]], [[113.93791546343081, 22.35787679672303], [114.01782410871618, 22.405519818125416]], [[113.99372501995832, 22.24961765932702], [114.02722641095893, 22.30719459127742]], [[113.98994934959073, 22.22015117902022], [113.99074033672426, 22.245450793734264]], [[114.12609255193942, 22.22779481895737], [114.16857788794387, 22.276267692594885]], [[114.01894097796169, 22.363194033357406], [114.08662893696763, 22.472785292961706]], [[114.10665147693905, 22.370883581921905], [114.23397426492734, 22.444538035154036]], [[114.14917290838618, 22.398823582262477], [114.22004450686822, 22.428574974772967]], [[114.18644026427476, 22.336579337296747], [114.30792942453161, 22.478985009295975]], [[113.95160102208226, 22.402849288997682], [114.2354947378698, 22.486510184588507]], [[113.97537760424841, 22.294854503751587], [114.08436646416386, 22.351235984632414]], [[113.98723988133435, 22.21911927811007], [114.18507300418617, 22.25667720914321]], [[113.98188329573911, 22.395706562963404], [114.10971940201871, 22.487066198960623]], [[113.95568041248178, 22.211418232611848], [114.089857876143, 22.298284390470503]], [[113.93626828030834, 22.20053767447553], [114.0011445510055, 22.215593239164907]], [[113.9428917626879, 22.222393874435785], [113.96657094151807, 22.26313706183892]], [[114.09647334292447, 22.1978520529682], [114.15567362854283, 22.279581319840627]], [[113.93127016342511, 22.24029238772986], [113.93133415616886, 22.260583593926818]], [[114.02321648511226, 22.250018214421843], [114.0959096264096, 22.474371132434126]], [[114.03771851971304, 22.297291378040043], [114.07242183379745, 22.39045230103068]]]
#print(len(random_points))
random_points+=[[[114.06068961768129, 22.20311825374936], [114.0685401057541, 22.243116292614456]], [[114.2263067677991, 22.50194523157252], [114.27256422844611, 22.504257943907444]], [[114.09662011266849, 22.201712551554362], [114.15295582081279, 22.21505791884449]], [[113.97728628907608, 22.290878881549364], [114.02748017711606, 22.36942144860932]], [[113.94149059284678, 22.36060959670548], [114.05131237089478, 22.481301553695854]], [[113.9399211336652, 22.372301642577664], [113.97755773588669, 22.491107363091345]], [[114.01419078106989, 22.208832878541543], [114.18822355061339, 22.20941607084293]], [[113.96246748521664, 22.19924415239848], [113.97407558917821, 22.327514291431314]], [[114.28178145342886, 22.28082796669083], [114.29888807398275, 22.299415790585716]], [[113.99004156037108, 22.267773024933838], [114.00486783933198, 22.320389142059653]]]
#print(len(random_points))
wQ=windowQuery()
#----------------------------query of approach 2 ------------------------------
record=[]
time_rec=[]
count=0
for item in random_points:
    #print("The ",count+1," query window is ",item)
    count+=1
    time_single_query=[]
    for i in range(5):
        wQ.RTree_query_res=0
        wQ.RTree_query_check_sum=0
        starttime=time.time()
        wQ.approach2(item, root)
        res, check_sum = wQ.RTree_query_res, wQ.RTree_query_check_sum
        #record.append(res)
        endtime=time.time()
        timespan=round(endtime - starttime, 5)
        time_single_query.append(timespan)
    record.append([res, check_sum])
    print("The number of objects in NO.", count," query window : ",res,', checked polygon number: ',check_sum)
    print("min runtime : ", min(time_single_query), ". max runtime : ", max(time_single_query), ". avg runtime : ", sum(time_single_query)/len(time_single_query), ".")
    #print("The number of polygons checked is ", check_sum)
    #break

The number of objects in NO. 1  query window :  494 , checked polygon number:  3894
min runtime :  0.00206 . max runtime :  0.00279 . avg runtime :  0.002248 .
The number of objects in NO. 2  query window :  2266 , checked polygon number:  2804
min runtime :  0.00194 . max runtime :  0.00324 . avg runtime :  0.002318 .
The number of objects in NO. 3  query window :  761 , checked polygon number:  1828
min runtime :  0.001 . max runtime :  0.00125 . avg runtime :  0.001088 .
The number of objects in NO. 4  query window :  2 , checked polygon number:  233
min runtime :  0.0001 . max runtime :  0.00015 . avg runtime :  0.00011 .
The number of objects in NO. 5  query window :  1793 , checked polygon number:  3602
min runtime :  0.00185 . max runtime :  0.00275 . avg runtime :  0.0021420000000000002 .
The number of objects in NO. 6  query window :  3916 , checked polygon number:  5821
min runtime :  0.00405 . max runtime :  0.00545 . avg runtime :  0.004542 .
The number of objects in NO. 7 